## Import libraries

In [1]:
import os, json, time
import gc
from IPython.display import display, Markdown
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import transformers
import torch
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM, TrainingArguments
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from langchain_community.document_loaders import TextLoader # new version
from langchain.prompts.prompt import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_community.vectorstores import FAISS, Chroma

# Text chunk spliter
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter , SentenceTransformersTokenTextSplitter# Text Splitter
from langchain.embeddings import HuggingFaceEmbeddings
from datasets import Dataset, DatasetDict, load_dataset

# for evaluate  LLM 
import evaluate
import pytest
import trulens
import wandb # for LLM  logging


2024-07-02 18:10:47.596411: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Configure

In [2]:
class CFG:
    OFFLINE = False # later we test on-premises evnironment use
    USE_WANDB = False # for LLM Evaluation and  Tracking
    ASR_MODEL = "openai/whisper-large-v3" # ASR model
    LLM_MODEL1 = "google/gemma-2b-it" # LLM model
    LLM_MODEL2 = "google/gemma-7b-it" # LLM model
    LLM_MODEL3 = "google/gemma-2-9b"# for new LLM model
    LLM_MODEL4 = "meta-llama/Meta-Llama-3-8B" # new LLama3 8B 

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
asr = AutoModelForSpeechSeq2Seq.from_pretrained(CFG.ASR_MODEL,
                                                 torch_dtype=torch.float16, 
                                                 low_cpu_mem_usage=True, 
                                                 use_safetensors=True
                                                ).to(device)
audioProc = AutoProcessor.from_pretrained(CFG.ASR_MODEL) 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Test ASR model

In [10]:
testspeechDataset1 = "hf-internal-testing/librispeech_asr_dummy"
testspeechDataset2 = "distil-whisper/librispeech_long"
audioDataset = load_dataset(testspeechDataset1, "clean", split="validation")

Generating validation split:   0%|          | 0/73 [00:00<?, ? examples/s]

In [11]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=asr,
    tokenizer=audioProc.tokenizer,
    feature_extractor=audioProc.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch.float16,
    device=device,
)

In [15]:
audioDataset[0]["audio"]

SystemError: initialization of _internal failed without raising an exception